<h1 style="padding-top: 25px;padding-bottom: 25px;text-align: left; padding-left: 10px; background-color: #DDDDDD; 
    color: black;"> <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science </h1>

## Detailed Examples: Data Collection, Parsing, and Quick Analyses

**Harvard University**<br/>
**Fall 2020**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader, and Chris Tanner<br/>
<hr style='height:2px'>

# Title

**Дополнительная практика + решения!**

# Описание

**Это упражнение не будет оцениваться, его даже нельзя сдать, и работать над ним совершенно необязательно**.

Но если вы хотите **дополнительной практики**, мы создали эту тетрадь, которая по своей сути очень похожа на домашнее задание. Более того, она очень похожа на реальные сценарии исследования и анализа данных - до моделирования.

Мы не включили автогрейдер, поэтому вы не сможете проверить свои решения. Однако мы предоставляем решения, чтобы вы могли вручную проверить, соответствуют ли ваши результаты нашим. **Решения видны на вкладке вверху (справа) в этом окне.**.**

In [315]:
## запустите эту ячейку, чтобы получить правильное форматирование
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2020-CS109A/master/themes/static/css/cs109.css").text
HTML(styles)

## Обзор 

Цель данного блокнота - получить дополнительную практику по получению, разбору, очистке и анализу данных. Поскольку для решения реальных задач часто требуется сбор информации из различных источников, в том числе и из Интернета, веб-скрепинг является весьма полезным навыком. 
Для этого мы будем собирать данные IMDb о самых высокооплачиваемых актерах и актрисах, извлекать различные ключевые точки данных и использовать PANDAS, чтобы научиться агрегировать эти данные полезным образом.

### Задачи обучения
- Начать работу с блокнотами Jupyter Notebooks, которые являются невероятно популярными, мощными и будут нашим средством программирования на протяжении всего курса CS109A и CS109B.
- Ознакомиться с методами поиска и использования данных из онлайн-источников.
- Приобрести опыт исследования и простого анализа данных.
- Ознакомиться с PANDAS как средством хранения и работы с данными.
- Чувствовать себя хорошо подготовленным к выполнению HW1.

### Примечания
- Упражнение **ответственный скраппинг**. Веб-серверы могут работать медленно или не реагировать на запросы, если они получают слишком много запросов от одного и того же источника за короткий промежуток времени. 

В своем коде используйте задержку между запросами в 2 секунды. Это поможет избежать блокировки целевого сайта - представьте, как обидно, когда это происходит. 

Раздел 1 этого домашнего задания предполагает сохранение отсканированных веб-страниц на локальной машине.


- Запросы на веб-скреппинг могут занимать несколько минут. В зависимости от проекта это может занять часы, дни или длиться бесконечно долго (Google ползает по всей Сети).
- При работе Jupyter Notebook сохраняет состояние памяти. Таким образом, порядок запуска ячеек имеет значение и играет решающую роль; 

в процессе разработки и тестирования кода можно легко допустить ошибки, основываясь на том, "когда" вы запускаете различные ячейки. 

**Перед выполнением каждого домашнего задания Jupyter Notebook обязательно перезагрузите Jupyter Notebook и запустите весь блокнот с нуля (т.е. "Kernel -> Restart & Run All")**.

In [316]:
# импортировать необходимые библиотеки
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup

# Оглавление

0. Практика работы с regex
I. Получение данных IMDb
  1. Получение данных сайта через `запросы`
  2. BeautifulSoup
  3. Получение url актера + зарплата
  4. Соскрести остальные данные
II. Загрузка и изучение данных
  1. Сохранение и загрузка данных с помощью Pandas
  2. Очистка данных (переименование столбцов + изменение типов)
  3. Нарезка и сортировка данных
  4. Вычисление сводной статистики (min, max, mean и т.д.)
  5. `pd.cut`, `df.groupby` и гистограммы
  6. Исследование зависимости возраста от зарплаты
  7. Исследование зависимости зарплаты от пола
  8. Изучение соотношения премий и пола
  9. Исследование соотношения премий и секса, часть II
  10. Исследование композиторских кредитов

## 0. Практика работы с regex

Умение соскабливать, разбирать и анализировать простые данные веб-сайта очень полезно в самых разных ситуациях. Здесь мы рассмотрим голосование в Сенате США по вопросу утверждения кандидата на должность окружного судьи США: https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=116&session=2&vote=00157.

Мы предоставляем скраппинг. Ваша задача состоит в следующем:
1. Написать BeautifulSoup для захвата секции 'vote by positon' для обоих вариантов "Yea" и "Nay".
2. Напишите regex для извлечения имени каждого сенатора из разделов "За" и "Против".


In [317]:
url = "https://www.senate.gov/legislative/LIS/roll_call_votes/vote1162/vote_116_2_00157.htm"

# proxies = {
#     'http': "http://37.120.147.90:8081",
#     'https': "https://37.120.147.90.66:8081"
# }
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}
response = requests.get(url, headers=headers, timeout=10)
response

<Response [403]>

Имена всех сенаторов

In [318]:
import re

from bs4 import BeautifulSoup

In [319]:
soup = BeautifulSoup(response.content, 'lxml')
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">
<!-- Akamaizer will replace all instances of the token "AKAMWAFREFID", --><!-- in meta/CONTENT or within <span>, --><!-- with the reference ID generated by the Web Application Firewall.  --><html>
<head>
<title>Senate.gov - Unauthorized</title>
<!-- disable redirect per meeting of 02 Apr 2018 ;; remove space in URL if re-enabled -->
<!-- meta HTTP-EQUIV="REFRESH" CONTENT="10; URL=http://www.senate.gov/pagelayout/general/one_item_and_teasers/waf.htm?REFID= AKAMWAFREFID" -->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<style type="text/css">
                body {margin: 0; background-image: url("/apology_objects/senatelogo.jpg");}
        </style>
</head>
<body>
<table style="height:25%; width: 100%; border: none;">
<tr>
<td align="CENTER" style="padding: 0;" valign="middle">
<table style="border: none;" width="50%">
<tr>
<td style="padding: 0;">
<center>
Sorry, a potential security risk was detected

1. Теги: 
   - `<div>` - это тег блочного элемента, используется для группировки и структурирования содержимого.
   - `<span>` - это тег строчного элемента, используется для обозначения отдельной части текста или обертки для других элементов.
   
2. Классы: 
   - `newspaperDisplay_3column` - это класс, примененный к `<div>` элементу. Классы используются для определения стилей в CSS или для работы с элементами с помощью JavaScript.
   - `contenttext` - это класс, примененный к `<span>` элементу. Он используется для определения стилей, связанных с текстом, в CSS или для работы с элементом с помощью JavaScript.

3. Блоки: 
   - `<div class="newspaperDisplay_3column">` - это блок, заданный с использованием `<div>` элемента и класса "newspaperDisplay_3column". Он группирует содержимое, разбивая его на три столбца.
   - `<span class="contenttext">` - это блок, заданный с использованием `<span>` элемента и класса "contenttext". Он обычно используется для определенного текста или для добавления дополнительных стилей к тексту.


Просмотрев код страницы. я поняла, что нужная мне информация находится в блоке div, строчном теге span, классе **contenttext**



In [320]:
# <code>

# <div class="newspaperDisplay_3column" bis_skin_checked="1">
# <span class="contenttext">

# НУЖНЫЙ КОНТЕНТ (имена и голоса всех сенаторов)

# </span>
# </div>

# </code>

In [321]:
# судя по всему предмет разговора, это вот этот заголовок
title = soup.title
print(title.text)


Senate.gov - Unauthorized


In [322]:
all_data = soup.find_all("span", class_ = 'contenttext')
# необрамленную массу инфы с тегами внутри выведем циклом
data_senators_cleaned = []
for item in all_data:
    data_senators_cleaned.append(item.text)
    print(item.text)
# или так (дословно, получить текст элемента name для элемента name в списке names и записать в список )
# data_senators_cleaned = [item.text for item in all_data]

Так вышло что это один элемент списка, (все имена можно получить по индексу 0) поэтому разобьем данные по \n

In [323]:
data_senators_cleaned = data_senators_cleaned[0].split('\n')
# на текущий момент это то что нам нужно

IndexError: list index out of range

In [ ]:
# просмотрим первые 4 элемента списка. думаю нужно 
# 1. сплит по пробелу, удалить запятую если она есть и  первый элемент в имя и так далее
data_senators_cleaned[:4]

['Alexander (R-TN), Not Voting',
 'Baldwin (D-WI), Nay',
 'Barrasso (R-WY), Yea',
 'Bennet (D-CO), Nay']

In [ ]:
# Теперь наша задача состоит в том чтобы получить только имена. вероятно нужно создать финальный список а еще лучше словарь, в который нужно положить сплитнутую часть каждого элемента по пробелу
# senators_name, initials, vote - наши ключи, словари это наш value
name = []
init = []
vote = []
voting_data = {'senators_name': name, 'initials': init, 'votes': vote}

for item in data_senators_cleaned:
    splited_item = item.split(',')
    if len(splited_item) == 2:
        senator_info = splited_item[0].split('(')
        senator_name = senator_info[0].strip()
        senator_initial = senator_info[1].replace(')', '').strip()
        senator_vote = splited_item[1].strip()

        name.append(senator_name)
        init.append(senator_initial)
        vote.append(senator_vote)
        # чтобы не вылазило ошибок, сделаем это более юзерфрендли
    else:
        print(f"Неверный формат элемента: {item}")

# Проверка результатов
print(voting_data)

Неверный формат элемента: 
Неверный формат элемента: 
{'senators_name': ['Alexander', 'Baldwin', 'Barrasso', 'Bennet', 'Blackburn', 'Blumenthal', 'Blunt', 'Booker', 'Boozman', 'Braun', 'Brown', 'Burr', 'Cantwell', 'Capito', 'Cardin', 'Carper', 'Casey', 'Cassidy', 'Collins', 'Coons', 'Cornyn', 'Cortez Masto', 'Cotton', 'Cramer', 'Crapo', 'Cruz', 'Daines', 'Duckworth', 'Durbin', 'Enzi', 'Ernst', 'Feinstein', 'Fischer', 'Gardner', 'Gillibrand', 'Graham', 'Grassley', 'Harris', 'Hassan', 'Hawley', 'Heinrich', 'Hirono', 'Hoeven', 'Hyde-Smith', 'Inhofe', 'Johnson', 'Jones', 'Kaine', 'Kennedy', 'King', 'Klobuchar', 'Lankford', 'Leahy', 'Lee', 'Loeffler', 'Manchin', 'Markey', 'McConnell', 'McSally', 'Menendez', 'Merkley', 'Moran', 'Murkowski', 'Murphy', 'Murray', 'Paul', 'Perdue', 'Peters', 'Portman', 'Reed', 'Risch', 'Roberts', 'Romney', 'Rosen', 'Rounds', 'Rubio', 'Sanders', 'Sasse', 'Schatz', 'Schumer', 'Scott', 'Scott', 'Shaheen', 'Shelby', 'Sinema', 'Smith', 'Stabenow', 'Sullivan', 'Tester

В этом коде мы проходимся по каждому элементу в data_senators_cleaned.

1. splited_item = item.split(',') разделяет элемент по запятым, создавая список из имени и голоса сенатора.
2. Затем мы проверяем длину разделенного элемента. Если длина равна 2, значит, имеем правильный формат элемента.
3. Далее мы разделяем имя сенатора и его инициалы, используя открывающую скобку '(' в списке senator_info.
4. Мы очищаем имя и инициалы, удаляя лишние пробелы вокруг них и удаляя закрывающую скобку ')' в инициалах.
5. Голос сенатора сохраняется в переменную senator_vote.
6. Мы добавляем имя, инициалы и голос в соответствующие списки name, init и vote.
7. В противном случае, если формат элемента неверный, мы выводим соответствующее сообщение.

Обратите внимание, что этот код ожидает, что data_senators_cleaned является списком строк, и элементы содержат имя и инициалы сенатора, разделенные открывающей скобкой '(', и голосом, разделенным запятой ','

ПОДРОБНЕЕ:
1. Создание пустых списков name, init и vote, а также словаря voting_data, где name, init и vote будут использоваться в качестве ключей, а пустые списки - в качестве значений для ключей.
2. Начало цикла for, где каждый элемент из списка data_senators_cleaned будет последовательно обрабатываться.
3. Разбиение элемента на две части по запятой с помощью split(','), чтобы получить список splited_item, содержащий имя сенатора и голос.
4. Проверка, что длина splited_item равна 2, что указывает на правильный формат элемента.
5. Внутри условного оператора if, разделение имени сенатора и его инициалов, используя открывающую скобку ( в списке senator_info. Распаковка имени и инициалов, аккуратное удаление скобок и удаление лишних пробелов с помощью методов strip() и replace().
6. Получение голоса сенатора из второй части элемента с помощью strip(), чтобы удалить лишние пробелы с начала и конца строки.
7. Добавление имени, инициалов и голоса в соответствующие списки name, init и vote с помощью методов append().
8. Если формат элемента неверный, в пункте else выводится сообщение с некорректным форматом элемента.
9. Конец цикла for, при этом все элементы исходного списка обработаны.
10. Вывод voting_data, словаря, содержащего списки имен, инициалов и голосов сенаторов.

Результатом выполнения данного кода будет словарь voting_data, в котором ключи senators_name, initials и votes будут содержать соответствующие списки с данными о сенаторах.

Пожалуйста, обратите внимание, что этот код предполагает определенный формат входных данных и ожидает, что элементы в data_senators_cleaned будут соответствовать этому формату. Если формат данных отличается, возможно, потребуется внести изменения в код для правильной обработки ваших данных.

In [ ]:
for i in range(len(vote)):
    if vote[i] == 'Yea':
        vote[i] = 'Yes'
    elif vote[i] == 'Nay':
        vote[i] = 'No'

In [ ]:
# for name_val, init_val, vote_val in zip(name, init, vote):
#     print(f"Name: {name_val}    Initials: {init_val}    Vote: {vote_val}")

In [ ]:
import pandas as pd
df = pd.DataFrame(voting_data)
df

,senators_name,initials,votes
0,Alexander,R-TN,Not Voting
1,Baldwin,D-WI,No
2,Barrasso,R-WY,Yes
3,Bennet,D-CO,No
4,Blackburn,R-TN,Not Voting
...,...,...,...
95,Warren,D-MA,No
96,Whitehouse,D-RI,No
97,Wicker,R-MS,Yes
98,Wyden,D-OR,No


In [ ]:
df[['votes']].value_counts()

votes     
Yes           55
No            42
Not Voting     3
dtype: int64

In [ ]:
df.to_csv('voting_data.csv', sep = ',', header = True, index=False)

2. Напишите regex для извлечения имени каждого сенатора из разделов "За" и "Против".

In [ ]:
# это решилось гораздо проще, но регулярки оставлю на потом

**Объяснение regex:** `'\)(.*?)\s\('`)

Я заметил, что имена были перечислены между скобками, например, `...)Barrasso (R...`. Поэтому я решил искать текст между скобками с пробелом перед открытыми скобками, т.е. `)abc (`. 

Регекс-поиск возвращает список всех совпадений со следующим условием: совпадает любая строка любой длины, которая идет после ")", но до " (".

![](https://i.imgur.com/5wiR1jn.png)

## 1. Получение данных IMDb

В данном случае нас интересует анализ нескольких точек данных об известных актерах и актрисах на сайте IMDb. IMDb предоставляет соответствующие данные, включающие имена, пол и различные награды актеров и актрис. Посетив сайт https://www.imdb.com/list/ls026028927/, вы найдете список самых высокооплачиваемых актеров и актрис. Каждый актер 

В этом упражнении мы сосредоточимся на автоматизации загрузки данных о каждом актере (с помощью ``запросов''). Сначала, как и для каждого Jupyter Notebook, импортируем необходимые пакеты, которые будем использовать в блокноте (т.е. запустим ячейку ниже)

In [ ]:
# Мы определяем это для удобства, так как URL каждого актера начинается с этого префикса
base_url = 'https://www.imdb.com'
extension = '/list/ls026028927/'

Здесь мы получаем веб -страницу и строим из нее объект BeautifulSoup (HTML -анализатор).

In [ ]:
actors_page = requests.get(base_url + extension)
bs_page = BeautifulSoup(actors_page.content, "html.parser")
bs_page


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>The World's Highest-Paid Actors And Actresses 2017 - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.com/list/ls026028927/" rel="canonical"/>
<meta content="http://www.imdb.com/list/ls026028927/" property="og:url">
<script>
    i

<div class='exercise'><b> Exercise 1.1:</b>
    
В ячейке ниже напишите код, который использует объект BeautifulSoup для разбора главной страницы, чтобы извлечь **ссылку и зарплату** для каждого актера. В частности, заполните словарь `info`, задав для каждого ключа имя актера, а для значения - **словарь данных**, с ключами `url` и `salary`. После завершения работы в словаре `info` будет 30 ключей. Например, внутри `info` одна из пар <ключ-значение> должна быть такой:

``'Adam Sandler': {'salary': '$50.5 million',
  'url': 'https://www.imdb.com/name/nm0001191'}``

**HINT:** Существует _множество_ решений, но вам может показаться, что проще всего использовать регулярное выражение (выражения)
</div>



все находится в тегe **div**, класс **lister-item-content** и актеры нумерованы

<!-- <span class="lister-item-index unbold text-primary">2. </span>
<a href="/name/nm0425005?ref_=nmls_hd"> Dwayne Johnson -->

имена находятся в тегах a

тект находтся в тегах p
<!-- <p>
Dwayne Douglas Johnson, also known as The Rock, was born on May 2, 1972 in Hayward, California. He is the son of <a href="/name/nm1696097?ref_=nmls_mkdn">Ata Johnson</a> (born Feagaimaleata Fitisemanu) and professional wrestler <a href="/name/nm0426117?ref_=nmls_mkdn">Rocky Johnson</a> (born Wayde Douglas Bowles). His father, from Amherst, Nova Scotia, Canada, is black (of Black Nova ...   
             </p> -->


для начала попробуем получить доступ к первому актеру

In [ ]:
first = bs_page.find("div", class_='lister-item-content')
first.text

'\n\n1. \n Mark Wahlberg\n \n\n                        Producer |\n The Fighter\n \n\nAmerican actor Mark Wahlberg is one of a handful of respected entertainers who successfully made the transition from teen pop idol to acclaimed actor. A Best Supporting Actor Oscar nominee for The Departed (2006) who went on to receive positive critical reviews for his performance in The Fighter (...                \n'

Окей, мы получили какие то данные по Mark Wahlberg. Но по видимому, я не захватила весь блок. смотрим:

In [ ]:
# <div class="lister-item mode-detail" bis_skin_checked="1">
#         <div class="lister-item-image" bis_skin_checked="1">
# <a href="/name/nm0000242/?ref_=nmls_pst"> <img alt="Mark Wahlberg" height="209" src="https://m.media-amazon.com/images/M/MV5BMTU0MTQ4OTMyMV5BMl5BanBnXkFtZTcwMTQxOTY1NA@@._V1_UY209_CR10,0,140,209_AL_.jpg" width="140">
# </a>        </div>
#         <div class="lister-item-content" bis_skin_checked="1">
#             <h3 class="lister-item-header">
#                 <span class="lister-item-index unbold text-primary">1. </span>
# <a href="/name/nm0000242?ref_=nmls_hd"> Mark Wahlberg
# </a>            </h3>
#                 <p class="text-muted text-small">
#                         Producer <span class="ghost">|</span>
# <a href="/title/tt0964517/?ref_=nmls_kf"> The Fighter
# </a>                </p>
#                 <p>
# American actor Mark Wahlberg is one of a handful of respected entertainers who successfully made the transition from teen pop idol to acclaimed actor. A Best Supporting Actor Oscar nominee for <a href="/title/tt0407887?ref_=nmls_mkdn">Отступники</a> (2006) who went on to receive ...                </p>
#         </div>
#         <div class="clear" bis_skin_checked="1"></div>
# <div class="list-description" bis_skin_checked="1"><p>$68 million</p></div>    </div>

In [ ]:
#  так лучше. где то в конце, мы все таки захватили заработную плату, значит класс мы сейчас определили верно
first = bs_page.find("div", class_='lister-item mode-detail')
first

<div class="lister-item mode-detail">
<div class="lister-item-image">
<a href="/name/nm0000242/"> <img alt="Mark Wahlberg" height="209" src="https://m.media-amazon.com/images/M/MV5BMTU0MTQ4OTMyMV5BMl5BanBnXkFtZTcwMTQxOTY1NA@@._V1_UY209_CR10,0,140,209_AL_.jpg" width="140"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1. </span>
<a href="/name/nm0000242"> Mark Wahlberg
</a> </h3>
<p class="text-muted text-small">
                        Producer <span class="ghost">|</span>
<a href="/title/tt0964517/"> The Fighter
</a> </p>
<p>
American actor Mark Wahlberg is one of a handful of respected entertainers who successfully made the transition from teen pop idol to acclaimed actor. A Best Supporting Actor Oscar nominee for <a href="/title/tt0407887">The Departed</a> (2006) who went on to receive positive critical reviews for his performance in <a href="/title/tt0964517">The Fighter</a> (...                </p

`info` - это словарь словарей, где ключи это Имена актеров и вложенные в них словари, с ключами `url` и `salary`

Вот так: 
``'Adam Sandler': {'salary': '$50.5 million',
  'url': 'https://www.imdb.com/name/nm0001191'}``

для url и зарплаты и правда нужно воспользоваться  регулярками, хотя ссылку попробуем извлечь по другому, плюс она состоит из 2 частей, поэтому придется мерджить

# Актер или актриса

In [ ]:
# h3 p text-muted text-small
credits = first.find('p', class_= 'text-muted text-small')
credits = credits.text.strip().split()[0]
credits

'Producer'

# ССылка

In [ ]:
#  <a href="/name/nm0000242/?ref_=nmls_pst"> <img alt="Mark Wahlberg" 
link_part2 = first.find('a')['href']
# окей, дернули из содержимого предыдущего поиска часть нашей ссылки
# конкатенируем с базой и проверяем
link = base_url+link_part2 
link
# отлично, ссылка рабочая, ведет куда нужно


'https://www.imdb.com/name/nm0000242/'

# Зарплата

In [ ]:
# <div class="list-description"><p>$68 million</p></div> </div>
salary = first.find('div', class_= 'list-description')
salary.text
# ура. С даступом мы разобрались


'$68 million'

# Имя

In [ ]:
# дословно найти в теге h3 классе lister-item-header тег а, из которого взять текст и на всякий случай сплитнуть пробелы
name = first.find('h3', class_='lister-item-header').find('a').text.strip()
name

'Mark Wahlberg'

# Теперь наша задача сделать так чтобы оно находило все записи через find all и проделывало то что мы делали вручную

In [ ]:
# получим доступ ко всем блокам всех актеров и сохраним их в список
actors_list_not_cleaned = bs_page.find_all("div", class_='lister-item mode-detail')
#  так мы все получили. теперь это список объектов по которым мы будем итерировать
actors_list_not_cleaned[2]


<div class="lister-item mode-detail">
<div class="lister-item-image">
<a href="/name/nm0004874/"> <img alt="Vin Diesel" height="209" src="https://m.media-amazon.com/images/M/MV5BMjExNzA4MDYxN15BMl5BanBnXkFtZTcwOTI1MDAxOQ@@._V1_UY209_CR5,0,140,209_AL_.jpg" width="140"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">3. </span>
<a href="/name/nm0004874"> Vin Diesel
</a> </h3>
<p class="text-muted text-small">
                        Actor <span class="ghost">|</span>
<a href="/title/tt2015381/"> Guardians of the Galaxy
</a> </p>
<p>
Vin Diesel was born Mark Sinclair in Alameda County, California, along with his fraternal twin brother, <a href="/name/nm0898731">Paul Vincent</a>. He was raised by his astrologer/psychologist mother, Delora Sherleen (Sinclair), and adoptive father, Irving H. Vincent, an acting instructor and theatre manager, in an artists' ...                </p>
</div>
<div class="clear"></di

In [ ]:
actors_names =  []
urls = []
salarys_millions = []
position_list= []
for item in actors_list_not_cleaned:
    # нам нужно имя
    name = item.find('h3', class_='lister-item-header').find('a').text.strip()
    actors_names.append(name)
    # нам нужно понять кто он такой и как себя позиционирует
    position = item.find('p', class_= 'text-muted text-small').text.strip().split()[0]
    position_list.append(position)
    # нам нужна ссылка
    link = base_url + item.find('a')['href']
    urls.append(link)
    # нам нужна зарплата
    salary = item.find('div', class_= 'list-description').text # зп
    salarys_millions.append(salary)
    
    

In [ ]:
print(actors_names)
print(urls)
print(salarys_millions)
print(position_list)

['Mark Wahlberg', 'Dwayne Johnson', 'Vin Diesel', 'Adam Sandler', 'Jackie Chan', 'Robert Downey Jr.', 'Tom Cruise', 'Shah Rukh Khan', 'Salman Khan', 'Akshay Kumar', 'Chris Hemsworth', 'Tom Hanks', 'Samuel L. Jackson', 'Ryan Gosling', 'Emma Stone', 'Jennifer Aniston', 'Jennifer Lawrence', 'Ryan Reynolds', 'Matt Damon', 'Jeremy Renner', 'Chris Evans', 'Melissa McCarthy', 'Chris Pratt', 'Mila Kunis', 'Emma Watson', 'Charlize Theron', 'Mark Ruffalo', 'Cate Blanchett', 'Julia Roberts', 'Amy Adams']
['https://www.imdb.com/name/nm0000242/', 'https://www.imdb.com/name/nm0425005/', 'https://www.imdb.com/name/nm0004874/', 'https://www.imdb.com/name/nm0001191/', 'https://www.imdb.com/name/nm0000329/', 'https://www.imdb.com/name/nm0000375/', 'https://www.imdb.com/name/nm0000129/', 'https://www.imdb.com/name/nm0451321/', 'https://www.imdb.com/name/nm0006795/', 'https://www.imdb.com/name/nm0474774/', 'https://www.imdb.com/name/nm1165110/', 'https://www.imdb.com/name/nm0000158/', 'https://www.imdb.co

Сразу пробежимся по списук зарплат удалим значки доллара и все что не цифры

In [ ]:
# Поступим как чайники и просто реплейснем все слова так как они одинаковы , ну и значки доллара
salarys_millions = [salary.replace('$', '').replace('million', '').strip() for salary in salarys_millions]
# Вывод очищенного спискаprint(salarys_cleaned)
# salarys_millions

# Так как нигде нет пола, будем хитрить. если Actor и Produser - male,  иначе female

In [ ]:
sex_list = []
for item in position_list:
    if item == 'Producer' or item == 'Actor':
        sex_list.append('male')
    else:
        sex_list.append('female')
        


# Время собрать все в словари

In [325]:
# actors_dict = {'actors_name': name, 'salary': salary, 'url': url}
# это будет наш список словарей
actors_data=[]
for name, sex, position, salary, url in zip(actors_names, sex_list, position_list, salarys_millions, urls):
    # добавить в список словарь переменным значениям которого будут присваиваться текущие значения итерируемых переменных
    actors_data.append({'actors_name': name, 'sex': sex, 'position': position,'salary': salary, 'url': url})

In [326]:
#  так как это список словарей, можем посмотреть на какой то конкретный элемент по индексу
actors_data[1]


{'actors_name': 'Dwayne Johnson',
 'sex': 'male',
 'position': 'Actor',
 'salary': '65',
 'url': 'https://www.imdb.com/name/nm0425005/'}

In [328]:
#  так как внутри у нас в любом элементе списка есть словарь, к его элементам можно обращаться по ключу
actors_data[1]['actors_name']

'Dwayne Johnson'

In [329]:
# проверим. красота
act_data = pd.DataFrame(actors_data)
act_data.head()

,actors_name,sex,position,salary,url
0,Mark Wahlberg,male,Producer,68,https://www.imdb.com/name/nm0000242/
1,Dwayne Johnson,male,Actor,65,https://www.imdb.com/name/nm0425005/
2,Vin Diesel,male,Actor,54.5,https://www.imdb.com/name/nm0004874/
3,Adam Sandler,male,Actor,50.5,https://www.imdb.com/name/nm0001191/
4,Jackie Chan,male,Actor,49,https://www.imdb.com/name/nm0000329/


# Усложним задачу. изучим краткое инфа по актеру и разберемся, что можно еще вытащить
у каждого актера есть минибио, которое находится по ссылке на него. нужно просмотреть все ли там одинаково, ибо есть еще одна чать ссылки

после нее доставляется третья часть `bio/?ref_=nm_ov_bio_sm`

In [330]:
# Забьем это в наш датафрейм на всякий случай)
act_data['short_bio_url'] = act_data['url']+'bio/?ref_=nm_ov_bio_sm'
act_data.head()

,actors_name,sex,position,salary,url,short_bio_url
0,Mark Wahlberg,male,Producer,68,https://www.imdb.com/name/nm0000242/,https://www.imdb.com/name/nm0000242/bio/?ref_=...
1,Dwayne Johnson,male,Actor,65,https://www.imdb.com/name/nm0425005/,https://www.imdb.com/name/nm0425005/bio/?ref_=...
2,Vin Diesel,male,Actor,54.5,https://www.imdb.com/name/nm0004874/,https://www.imdb.com/name/nm0004874/bio/?ref_=...
3,Adam Sandler,male,Actor,50.5,https://www.imdb.com/name/nm0001191/,https://www.imdb.com/name/nm0001191/bio/?ref_=...
4,Jackie Chan,male,Actor,49,https://www.imdb.com/name/nm0000329/,https://www.imdb.com/name/nm0000329/bio/?ref_=...


**На каждой странице есть краткое Overview**

Born June 5, 1971 · Dorchester, Boston, Massachusetts, USA

Birth name Mark Robert Michael Wahlberg

Nicknames Marky MarkMonk D

Height 1.73 m

***В конце состав семьи и иное ***

**Посмотрим сначала на Overview, который на странице выглядит примерно так (структура не сохранена)**

In [ ]:
<div data-testid="sub-section-overview" class="sc-f65f65be-0 fVkLRr" bis_skin_checked="1"><ul class="ipc-metadata-list ipc-metadata-list--dividers-between sc-c1dc6938-0 czYoHO meta-data-list-full ipc-metadata-list--base" role="presentation">
<li role="presentation" class="ipc-metadata-list__item" id="born" data-testid="list-item"><span class="ipc-metadata-list-item__label" aria-disabled="false">Born</span><div class="ipc-metadata-list-item__content-container" bis_skin_checked="1">
<div class="ipc-html-content ipc-html-content--base ipc-html-content--display-inline" role="presentation" bis_skin_checked="1">
<div class="ipc-html-content-inner-div" bis_skin_checked="1"><a href="/search/name/?birth_monthday=6-5&amp;refine=birth_monthday&amp;ref_=nmbio_ov_born_monthday">June 5</a>, 
<a href="/search/name/?birth_year=1971&amp;ref_=nmbio_ov_born_year">1971</a> · <a href="/search/name/?birth_place=Dorchester, Boston, Massachusetts, USA&amp;ref_=nmbio_ov_born_location">Dorchester, Boston, Massachusetts, USA</a>
</div></div></div></li><li role="presentation" class="ipc-metadata-list__item" id="name" data-testid="list-item"><span class="ipc-metadata-list-item__label" aria-disabled="false">Birth name</span><div class="ipc-metadata-list-item__content-container" bis_skin_checked="1"
<div class="ipc-html-content ipc-html-content--base ipc-html-content--display-inline" role="presentation" bis_skin_checked="1"><div class="ipc-html-content-inner-div" bis_skin_checked="1">Mark Robert Michael Wahlberg</div>
</div></div></li><li role="presentation" class="ipc-metadata-list__item" id="nicknames" data-testid="list-item"><span class="ipc-metadata-list-item__label" aria-disabled="false">Nicknames</span><div class="ipc-metadata-list-item__content-container" bis_skin_checked="1">
<ul class="ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content base" role="presentation"><li role="presentation" class="ipc-inline-list__item"><span class="ipc-metadata-list-item__list-content-item" aria-disabled="false">Marky Mark</span></li>
<li role="presentation" class="ipc-inline-list__item"><span class="ipc-metadata-list-item__list-content-item" aria-disabled="false">Monk D</span></li></ul><div class="ipc-html-content ipc-html-content--base ipc-html-content--display-inline" role="presentation" bis_skin_checked="1"></div>
</div></li><li role="presentation" class="ipc-metadata-list__item" id="height" data-testid="list-item"><span class="ipc-metadata-list-item__label" aria-disabled="false">Height</span><div class="ipc-metadata-list-item__content-container" bis_skin_checked="1">
<div class="ipc-html-content ipc-html-content--base ipc-html-content--display-inline" role="presentation" bis_skin_checked="1"><div class="ipc-html-content-inner-div" bis_skin_checked="1">1.73 m</div></div></div></li></ul></div>

In [331]:
# получим контент нужного элемента пока одного конкатенируя ссылку со словаря которые мы спарсили с третьей частью
Mark_Wahlberg_page = requests.get(urls[0]+'bio/?ref_=nm_ov_bio_sm')
MW_bs_page = BeautifulSoup(Mark_Wahlberg_page.content, "html.parser")
MW_bs_page
# получаем 403. чиним


<html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
</body>
</html>

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}
Mark_Wahlberg_page = requests.get(urls[0]+'bio/?ref_=nm_ov_bio_sm', headers = headers)
MW_bs_page = BeautifulSoup(Mark_Wahlberg_page.content, "html.parser")
MW_bs_page
# отлично

<div class='exercise'><b> Exercise 1.2:</b>
    
В ячейке ниже напишите код, который использует объект BeautifulSoup для разбора url каждого актера, чтобы извлечь эти дополнительные переменные и добавить их в `info`:
- birth date
- sex
- height
- producer credits
- actor credits
- soundtrack credits
- composer credits
- award wins
- award nominations

Сохраните это в виде фрейма данных Pandas с именем `df`, где каждая строка представляет конкретного актера. Проверьте `df.shape`: у вас должно быть 30 строк и 11 столбцов.

**HINT:** Чтобы получить информацию о победах и номинациях, вы можете добавить `'/awards'` к вашему url и запросить эту ссылку. Не забывайте об ответственном подходе к порсингу!

**HINT:** Актерские кредиты перечислены в разделе "Актер" для мужчин и "Актриса" для женщин. Убедитесь, что вы спарсили правильно.

</div>

In [356]:
# Найти элемент со всей необходимой информацией
section_element = MW_bs_page.find('section', class_='ipc-page-section--base')
section_element.text

'OverviewBornJune 5, 1971 · Dorchester, Boston, Massachusetts, USABirth nameMark Robert Michael WahlbergNicknamesMarky MarkMonk DHeight1.73 m'

In [336]:
# # Вся нужная нам информация о рождении есть в теге  div, класса ipc-html-content-inner-div, теге а href
# block_info = MW_bs_page.find('div', class_='ipc-html-content-inner-div').find_all('a')
# for item in block_info:
#     print(item.text)
# 

In [337]:
# Найти элемент с датой рождения (<li role="presentation" class="ipc-metadata-list__item" id="born" data-testid="list-item">)
born_element = section_element.find('li', id='born')
# найдем все теги а
born_date_place = born_element.find_all('a')
# проитерируем по стписку в котором хранятся все теги а этого блока и сохраним его в список 
dates_and_place = [date.text for date in born_date_place]
# проверка
dates_and_place

['June 5', '1971', 'Dorchester, Boston, Massachusetts, USA']

In [353]:

# Найти элемент с именем при рождении
birth_name_element = section_element.find('li', id='name')
birth_name = birth_name_element.find('div', class_='ipc-html-content-inner-div').text
birth_name

'Mark Robert Michael Wahlberg'

In [345]:
# Найти элементы с никнеймами
nicknames_element = section_element.find('li', id='nicknames')
nicknames_list = nicknames_element.find_all('span', class_='ipc-metadata-list-item__list-content-item')
nicknames = [nickname.text for nickname in nicknames_list]
nicknames

['Marky Mark', 'Monk D']

In [340]:
# Найти элемент с ростом
height_element = section_element.find('li', id='height')
height = height_element.find('div', class_='ipc-html-content-inner-div').text

In [341]:
# Вывести найденную информацию
print("Дата рождения:", dates_and_place)
print("Имя при рождении:", birth_name)
print("Никнеймы:", nicknames)
print("Рост:", height)

Дата рождения: ['June 5', '1971', 'Dorchester, Boston, Massachusetts, USA']
Имя при рождении: Mark Robert Michael Wahlberg
Никнеймы: ['Marky Mark', 'Monk D']
Рост: 1.73 m


In [343]:
# напоминашка
act_data.head(1)

,actors_name,sex,position,salary,url,short_bio_url
0,Mark Wahlberg,male,Producer,68,https://www.imdb.com/name/nm0000242/,https://www.imdb.com/name/nm0000242/bio/?ref_=...


# Добавим по всем актерам:

In [380]:
birth_date_list = []
bitrh_year_list = []
place_of_birth_list = []

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}
# наш список urls по нему нужно итерировать получая и итератор (индекс) и сам элемент
# когда страница конкретного url извлечена, сразу парсим оттуда все необходимые данные 
for index, page in enumerate(urls):
    current_page = requests.get(urls[index]+'bio/?ref_=nm_ov_bio_sm', headers = headers)
    page = BeautifulSoup(current_page.content, "html.parser")
    # найдем секцию
    section = page.find('section', class_='ipc-page-section--base')
    # Найти элемент с датой рождения (<li role="presentation" class="ipc-metadata-list__item" id="born" data-testid="list-item">)
    born_element = section.find('li', id='born')
    # найдем все теги а
    born_date_place = born_element.find_all('a')
    # проитерируем по стписку в котором хранятся все теги а этого блока и сохраним его в список 
    dates_and_place = [date.text for date in born_date_place]
    # формат ['June 5', '1971', 'Dorchester, Boston, Massachusetts, USA']
    birth_date_list.append(dates_and_place[0])
    bitrh_year_list.append(dates_and_place[1])  
    place_of_birth_list.append(dates_and_place[2])
    # Найти элемент с именем при рождении
 


# Тестовая ячейка

In [364]:
# headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
#     }
# test_page = requests.get(urls[3]+'bio/?ref_=nm_ov_bio_sm', headers = headers)
# response = BeautifulSoup(test_page.content, "html.parser")
#      # отлично
#     # Найти элемент со всей необходимой информацией
# section_element = response.find('section', class_='ipc-page-section--base')
#     # Найти элемент с именем при рождении
# birth_name_element = section_element.find('li', id='name')
# birth_name = birth_name_element.find('div', class_='ipc-html-content-inner-div').text
# # birth_name_list.append(birth_name)
    

#     # Найти элементы с никнеймами
# nicknames_element = section_element.find('li', id='nicknames')
# nicknames_list = nicknames_element.find_all('span', class_='ipc-metadata-list-item__list-content-item')
# nicknames = [nickname.text for nickname in nicknames_list]
# # all_nicknames_list.append(nicknames)
   

#     # Найти элемент с ростом
# height_element = section_element.find('li', id='height')
# height = height_element.find('div', class_='ipc-html-content-inner-div').text
# # height_list.append(height)
#     # Вывести найденную информацию
# print("Дата рождения:", dates_and_place)
# print("Имя при рождении:", birth_name)
# print("Никнеймы:", nicknames)
# print("Рост:", height)

Дата рождения: ['November 2', '1965', 'New Delhi, India']
Имя при рождении: Adam Richard Sandler
Никнеймы: ['Sandman']
Рост: 1.77 m


In [381]:
birth_name_list = []
all_nicknames_list =[]
height_list = []
for index in range(len(urls)):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    }
    test_page = requests.get(urls[index]+'bio/?ref_=nm_ov_bio_sm', headers=headers)
    response = BeautifulSoup(test_page.content, "html.parser")
       # Найти элемент со всей необходимой информацией
    section_element = response.find('section', class_='ipc-page-section--base')
    # Найти элемент с именем при рождении
    birth_name_element = section_element.find('li', id='name')
    # прикол в том, что вероятно не у всех есть никнеймы, в этом нужно убедится, но оно сильно ругается на find all()
    if birth_name_element:
        birth_name_div = birth_name_element.find('div', class_='ipc-html-content-inner-div')
        if birth_name_div:
            birth_name = birth_name_div.text
        else:
            birth_name = ''
    else:
        birth_name = ''
    birth_name_list.append(birth_name)
        

        # Найти элементы с никнеймами
    nicknames_element = section_element.find('li', id='nicknames')
    if nicknames_element:
        nicknames_list = nicknames_element.find_all('span', class_='ipc-metadata-list-item__list-content-item')
        nicknames = [nickname.text for nickname in nicknames_list]
        all_nicknames_list.append(nicknames)
    else:
        all_nicknames_list.append([])
    

        # Найти элемент с ростом
    height_element = section_element.find('li', id='height')
    height = height_element.find('div', class_='ipc-html-content-inner-div').text
    height_list.append(height)

# Добавим все в таблицу


In [390]:
act_data.drop('birth_date', axis=1, inplace=True)

In [393]:
act_data.head(2)

,actors_name,sex,position,salary,url,short_bio_url
0,Mark Wahlberg,male,Producer,68,https://www.imdb.com/name/nm0000242/,https://www.imdb.com/name/nm0000242/bio/?ref_=...
1,Dwayne Johnson,male,Actor,65,https://www.imdb.com/name/nm0425005/,https://www.imdb.com/name/nm0425005/bio/?ref_=...


In [397]:
print(len(birth_date_list))
print(len(bitrh_year_list))
print(len(place_of_birth_list))
print(len(birth_name_list))
print(len(height_list))
print(len(all_nicknames_list))
print(len(urls)) 
     

30
30
30
30
30
30
30


In [398]:
act_data.shape

(30, 6)

In [401]:
# act_data.drop('birth_date', axis=1, inplace=True)
# act_data.drop('birth_year', axis=1, inplace=True)
# act_data.drop('place_of_birth', axis=1, inplace=True)

In [402]:
for i, (birth_date, birth_year, place_of_birth) in enumerate(zip(birth_date_list, bitrh_year_list, place_of_birth_list)):
    act_data.at[i, 'birth_date'] = birth_date
    act_data.at[i, 'birth_year'] = birth_year
    act_data.at[i, 'place_of_birth'] = place_of_birth
    

или

In [ ]:
# for i, (birth_date, birth_year, place_of_birth) in enumerate(zip(birth_date_list, bitrh_year_list, place_of_birth_list)):
#     act_data.loc[i, ['birth_date', 'birth_year', 'place_of_birth']] = birth_date, birth_year, place_of_birth

In [403]:
act_data.head()

,actors_name,sex,position,salary,url,short_bio_url,birth_date,birth_year,place_of_birth
0,Mark Wahlberg,male,Producer,68,https://www.imdb.com/name/nm0000242/,https://www.imdb.com/name/nm0000242/bio/?ref_=...,June 5,1971,"Dorchester, Boston, Massachusetts, USA"
1,Dwayne Johnson,male,Actor,65,https://www.imdb.com/name/nm0425005/,https://www.imdb.com/name/nm0425005/bio/?ref_=...,May 2,1972,"Hayward, California, USA"
2,Vin Diesel,male,Actor,54.5,https://www.imdb.com/name/nm0004874/,https://www.imdb.com/name/nm0004874/bio/?ref_=...,July 18,1967,"Alameda County, California, USA"
3,Adam Sandler,male,Actor,50.5,https://www.imdb.com/name/nm0001191/,https://www.imdb.com/name/nm0001191/bio/?ref_=...,September 9,1966,"Brooklyn, New York City, New York, USA"
4,Jackie Chan,male,Actor,49,https://www.imdb.com/name/nm0000329/,https://www.imdb.com/name/nm0000329/bio/?ref_=...,April 7,1954,"Victoria Peak, Hong Kong"


In [405]:
for i, (birth_name, height) in enumerate(zip(birth_name_list, height_list)):
    act_data.at[i, 'birth_name'] = birth_name
    act_data.at[i, 'height'] = height
    


In [408]:
act_data.head()

,actors_name,sex,position,salary,url,short_bio_url,birth_date,birth_year,place_of_birth,birth_name,height
0,Mark Wahlberg,male,Producer,68,https://www.imdb.com/name/nm0000242/,https://www.imdb.com/name/nm0000242/bio/?ref_=...,June 5,1971,"Dorchester, Boston, Massachusetts, USA",Mark Robert Michael Wahlberg,1.73 m
1,Dwayne Johnson,male,Actor,65,https://www.imdb.com/name/nm0425005/,https://www.imdb.com/name/nm0425005/bio/?ref_=...,May 2,1972,"Hayward, California, USA",Dwayne Douglas Johnson,1.96 m
2,Vin Diesel,male,Actor,54.5,https://www.imdb.com/name/nm0004874/,https://www.imdb.com/name/nm0004874/bio/?ref_=...,July 18,1967,"Alameda County, California, USA",Mark Sinclair,1.82 m
3,Adam Sandler,male,Actor,50.5,https://www.imdb.com/name/nm0001191/,https://www.imdb.com/name/nm0001191/bio/?ref_=...,September 9,1966,"Brooklyn, New York City, New York, USA",Adam Richard Sandler,1.77 m
4,Jackie Chan,male,Actor,49,https://www.imdb.com/name/nm0000329/,https://www.imdb.com/name/nm0000329/bio/?ref_=...,April 7,1954,"Victoria Peak, Hong Kong",Kong-sang Chan,1.72 m


In [439]:
# Приводим пустые списки к значению NaN
all_nicknames_list = [nicknames if nicknames else np.nan for nicknames in all_nicknames_list]

# Добавляем список в столбец nicknames' в act_data
act_data['nicknames'] = all_nicknames_list

In [436]:
# act_data.drop('nicknames', axis=1, inplace=True)

In [440]:
act_data.head()

,actors_name,sex,position,salary,url,short_bio_url,birth_date,birth_year,place_of_birth,birth_name,height,nicknames
0,Mark Wahlberg,male,Producer,68,https://www.imdb.com/name/nm0000242/,https://www.imdb.com/name/nm0000242/bio/?ref_=...,June 5,1971,"Dorchester, Boston, Massachusetts, USA",Mark Robert Michael Wahlberg,1.73 m,"[Marky Mark, Monk D]"
1,Dwayne Johnson,male,Actor,65,https://www.imdb.com/name/nm0425005/,https://www.imdb.com/name/nm0425005/bio/?ref_=...,May 2,1972,"Hayward, California, USA",Dwayne Douglas Johnson,1.96 m,"[The People's Champion, The Brahma Bull, The G..."
2,Vin Diesel,male,Actor,54.5,https://www.imdb.com/name/nm0004874/,https://www.imdb.com/name/nm0004874/bio/?ref_=...,July 18,1967,"Alameda County, California, USA",Mark Sinclair,1.82 m,NaN
3,Adam Sandler,male,Actor,50.5,https://www.imdb.com/name/nm0001191/,https://www.imdb.com/name/nm0001191/bio/?ref_=...,September 9,1966,"Brooklyn, New York City, New York, USA",Adam Richard Sandler,1.77 m,[Sandman]
4,Jackie Chan,male,Actor,49,https://www.imdb.com/name/nm0000329/,https://www.imdb.com/name/nm0000329/bio/?ref_=...,April 7,1954,"Victoria Peak, Hong Kong",Kong-sang Chan,1.72 m,"[Y'uen Lo, Sing Lung, Pao Pao, Kung-Fu Master]"


# Награды

In [ ]:
# взяли какую то часть информации. поишем дальше. Награды
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}
rewards_page = requests.get(urls[0]+'awards', headers = headers)
parce_page = BeautifulSoup(rewards_page.content, "html.parser")
parce_page



In [ ]:
# блок обернут в <div class="sc-c7c3a435-1 NixYx ipc-page-grid__item ipc-page-grid__item--span-2"
# дальше идут секции <section class="ipc-page-section ipc-page-section--base">
# в секциях нам интересен текст <h3 class="ipc-title__text"><span id="ev0000003">Academy Awards, USA</span>
section_element_rewards = parce_page.find('div', class_='sc-c7c3a435-1 NixYx ipc-page-grid__item ipc-page-grid__item--span-2')
section_element_rewards
h3_list = section_element_rewards.find_all('h3',class_='ipc-title__text')
nomonation_list =[list.text for list in h3_list]
nomonation_list


['Academy Awards, USA',
 'Primetime Emmy Awards',
 'BAFTA Awards',
 'Satellite Awards',
 'Razzie Awards',
 'News & Documentary Emmy Awards',
 'International Documentary Association',
 'PGA Awards',
 "People's Choice Awards, USA",
 'Black Reel Awards for Television',
 'Realscreen Awards',
 'Teen Choice Awards',
 'Christopher Awards',
 '20/20 Awards',
 'Jupiter Award',
 'National Board of Review, USA',
 "Kids' Choice Awards, USA",
 'Critics Choice Awards',
 'MTV Movie + TV Awards',
 'CinEuphoria Awards',
 'Rembrandt Awards',
 'Monte-Carlo TV Festival',
 'Golden Globes, USA',
 'Screen Actors Guild Awards',
 'Central Ohio Film Critics Association',
 'Palm Springs International Film Festival',
 'Gold Derby Awards',
 'Washington DC Area Film Critics Association Awards',
 'Boston Society of Film Critics Awards',
 'San Diego Film Critics Society Awards',
 'Walk of Fame',
 'African-American Film Critics Association (AAFCA)',
 'Awards Circuit Community Awards',
 'Online Film Critics Society Awar

In [ ]:
# div ipc-signpost__text - количество наград
rewards_and_nominstions = section_element_rewards.find('div', class_='ipc-signpost__text').text
rewards_and_nominstions

'24 wins & 82 nominations'

Добьем наш датафрейм.нам необходима вся эта же информация, только по всем актерам

- birth date *done*
- sex *done*
- height *done*
- award wins *done*
- award nominations *done*
- award nominations names *done*

# осталось найти в разделе https://www.imdb.com/name/nm0000242/fullcredits/?ref_=nmawd_ql_flmg_6
- producer credits
- actor credits
- soundtrack credits
- composer credits

## 2. Loading and Exploring Data
Now, let's actually use the data! Here, we ask you to perform a few operations using PANDAS on our new dataset.

<div class='exercise'><b> Exercise 2.1: Saving & Loading Data</b>
    
Save the dataframe as a csv, and then load it back again, saving it under a different name. Compare the number of columns of your new dataframe to `df` above. If it is different, why?
</div>

In [ ]:
# YOUR CODE HERE
# END OF YOUR CODE HERE

The newly loaded dataframe turned the index of `df` into a new Unnamed column.

<div class='exercise'><b> Exercise 2.2: Cleaning Data</b>

First, to clean up the data frame, we will rename column headers that contain a space, i.e. changing "actor credits" to "actor_credits". This is standard practice and will make references to column names easier in the future.

Also, change all numerical columns to be of type `"int"` or `"float"`, whichever is more appropriate, by using the `astype` function.
</div>

In [ ]:
# YOUR CODE HERE
# END OF YOUR CODE HERE

<div class='exercise'><b> Exercise 2.3: Slicing & Sorting Data</b>

Use `df` for all future exercises. 

Create a subset of `df` of female actresses with more than 50 wins. Sort it by actor credits, then producer credits. So, if there is a tie in actor credits, the person with more producer credits should come 1st. Sort in decreasing order (most actor credits goes first).
</div>

In [ ]:
# YOUR CODE HERE
# END OF YOUR CODE HERE

<div class='exercise'><b> Exercise 2.4: Data Summary Statistics</b>

Calculate summary statistics for the actors' salaries and total credits (actor + producer + composer + soundtrack).

</div>

In [ ]:
# YOUR CODE HERE
# END OF YOUR CODE HERE

<div class='exercise'><b> Exercise 2.5: Group By</b>

Use the pandas `cut` function to group the actors by total credits into bins of 50 (i.e. 1-50, 51-100, etc.). Then use the `groupby` function to find the mean number of nominations for each bin and report it in a plot.

</div>

In [ ]:
# YOUR CODE HERE
# END OF YOUR CODE HERE

<div class='exercise'><b> Exercise 2.6: Exploring Age vs Salary</b>

One interesting question is looking at age vs salary : Is there a peak age that acquires the highest salary?; Are older actors paid less? In order to answer these questions, we must first extract age from the birthdate. We can do this by converting the birthdate to a datetime object.

More info on datetime objects: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html .

</div>

In [ ]:
# YOUR CODE HERE
# END OF YOUR CODE HERE

Now we explore the age statistics.

In [ ]:
# YOUR CODE HERE
# END OF YOUR CODE HERE

In [ ]:
# find the actor/actress associated with oldest age
# YOUR CODE HERE
# END OF YOUR CODE HERE

**Observe the results.** What do you notice about the two youngest actors/actresses? What do you notice about the oldest? 

The youngest are both female, and have significantly less credits than the oldest, who happens to be male. But notice that Jennifer Lawrence does have more wins than Emma Watson and Samuel Jackson. Do you think that it makes sense that more wins/awards correlates with a higher salary?

Let's look a little further in depth in the age range by splitting up age by quartiles according to the data. We can do this with pandas' built in .describe() function.

In [ ]:
df["age"].describe()

NameError: name 'df' is not defined

We the bin age groups based on these quartile summary statistics.

In [ ]:
quartile_1 = df[df.age <= 41]
quartile_2 = df[(df.age > 41) & (df.age <=49)]
quartile_3 = df[(df.age >49) & (df.age <=53)]
quartile_4 = df[df.age >53]

In [ ]:
# look at mean salary within each age quartile
# YOUR CODE HERE
# END OF YOUR CODE HERE

We can see that quartile 2 (ages 42 - 49) has higher average salary than quartile 3 (ages 50 - 53), but quartile 4 still has highest average salary (ages 54 - 71). Why might this be?

<div class='exercise'><b> Exercise 2.7: Exploring Salary vs sex</b>

Now, lets look at salary vs sex. This could be useful to look into any possible gender bias.

</div>

In [ ]:
# salary vs sex
# look at mean salary among male and females
# YOUR CODE HERE
# END OF YOUR CODE HERE

In [ ]:
# look at min and max salary among male and females
# YOUR CODE HERE
# END OF YOUR CODE HERE

What do you notice from these statistics? It looks like females have a lower salary in all three summary statistics. Why do you think this is? Do you think there are other factors that could be affecting this? If so, what else in the data could be indicative?

It could also be helpful to additionally look the average total number of credits per gender.

In [ ]:
# look at mean salary among male and females
# YOUR CODE HERE
# END OF YOUR CODE HERE

In [ ]:
# look at min and max salary among male and females
# YOUR CODE HERE
# END OF YOUR CODE HERE

Notice that although females have lower average salary than males, they also tend to have fewer credits. Does this tell us something about how the number of credits correlates with salary? How could you explore this further?

<div class='exercise'><b> Exercise 2.8: Exploring awards by sex</b>

Which actress has the most wins? How many wins does she have? Which actor has the most wins? How many wins does he have?

</div>

In [ ]:
# YOUR CODE HERE
# END OF YOUR CODE HERE

<div class='exercise'><b> Exercise 2.9: Exploring awards by sex, Part II</b>

What is the mean number of wins for an actor/actress with less than 65 actor credits? With greater than or equal to 65 actor credits?

</div>

In [ ]:
# YOUR CODE HERE
# END OF YOUR CODE HERE

<div class='exercise'><b> Exercise 2.10: Exploring composer credits</b>

How many actors/actresses have at least 1 composer credit? 

</div>

In [ ]:
# YOUR CODE HERE
# END OF YOUR CODE HERE